In [23]:
import requests
import os
from IPython.display import display
import pandas as pd

file = "11-15-2020.csv"

if not os.path.exists(file):
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/11-15-2020.csv"
    r = requests.get(url)
    with open(file, 'wb') as f:
        f.write(r.content)

df = pd.read_csv(file)
display(df.head())
display(df.describe())

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-16 05:25:57,33.93911,67.709953,43240,1617,35092,6531.0,Afghanistan,111.075887,3.739593
1,NaN,NaN,NaN,Albania,2020-11-16 05:25:57,41.15330,20.168300,27830,623,12889,14318.0,Albania,967.058169,2.238591
2,NaN,NaN,NaN,Algeria,2020-11-16 05:25:57,28.03390,1.659600,67679,2154,44633,20892.0,Algeria,154.338404,3.182671
3,NaN,NaN,NaN,Andorra,2020-11-16 05:25:57,42.50630,1.521800,5872,76,4747,1049.0,Andorra,7599.818805,1.294278
4,NaN,NaN,NaN,Angola,2020-11-16 05:25:57,-11.20270,17.873900,13451,322,6444,6685.0,Angola,40.926460,2.393874


,FIPS,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio
count,3263.000000,3889.000000,3889.000000,3.974000e+03,3974.000000,3.974000e+03,3.972000e+03,3889.000000,3931.000000
mean,32405.273675,36.005269,-71.993140,1.371061e+04,331.943634,8.797361e+03,4.581937e+03,3198.273805,1.985763
std,18007.161500,12.935577,53.497468,7.337294e+04,2027.361304,8.752082e+04,7.889037e+04,2173.424277,3.483595
min,66.000000,-52.368000,-174.159600,0.000000e+00,0.000000,0.000000e+00,-4.174884e+06,0.000000,0.000000
25%,19052.000000,33.272157,-96.580954,3.522500e+02,4.000000,0.000000e+00,3.010000e+02,1591.420170,0.772268
50%,30069.000000,37.952303,-86.853681,1.037000e+03,16.000000,0.000000e+00,9.080000e+02,2977.631096,1.501619
75%,47040.000000,42.209345,-77.492245,4.207500e+03,70.000000,0.000000e+00,2.938500e+03,4363.614738,2.538429
max,99999.000000,71.706900,178.065000,1.983480e+06,45974.000000,4.174884e+06,1.820670e+06,18559.970155,162.820368


In [24]:
df_sha256 = df.copy()

import hashlib

def hash_row(row: pd.Series) -> str:
    return hashlib.sha256(
            row.to_json().encode("utf-8")
        ).hexdigest()

%timeit df_sha256["hash"] = df_sha256.apply(hash_row, axis=1)

df_sha256["hash"].head()

184 ms ± 5.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


0    ce5c7227cdd7407b56bec84048c21b2d8c6b8a60d5d4f5...
1    3a67c675cc49761ca4bbdf47a0e5b4f332dfa4ca7be62b...
2    659a2d1b7a4e104ced0abb9fb1ce15cdd4954b85e1cb1c...
3    1ed856d5956a0cad75afdabb7312d54346ab09f0977519...
4    e9038ccfbf161bd54d830bb512782fc9ed57d65e022990...
Name: hash, dtype: object

In [25]:
df_pd_hash = df.copy()

%timeit df_pd_hash["hash"] = pd.util.hash_pandas_object(df_pd_hash)

df_pd_hash["hash"].head()

13 ms ± 656 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


0     462126260668559019
1    3661975257962623256
2    2087982810377571855
3    6476561374635745019
4      58280379837277900
Name: hash, dtype: uint64

In [26]:
df_xxhash = df.copy()

import xxhash

def hash_row(row: pd.Series) -> str:
    return xxhash.xxh64_hexdigest(
            row.to_json().encode("utf-8")
        )

%timeit df_xxhash["hash"] = df_xxhash.apply(hash_row, axis=1)

df_xxhash["hash"].head()

169 ms ± 5.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


0    30f63bd2e271b5cc
1    31496c54cc416705
2    5e7b53253ef8acac
3    b00f81bca1b3f47d
4    8c89a9e722b5a99b
Name: hash, dtype: object